In [1]:
import gzip,os,tarfile,sys
sys.path.append(os.pardir+'/src')
from settings import *
import pandas as pd
from pandas import DataFrame
import time
import numpy as np
import time
%matplotlib inline 

In [2]:
#　全ファイル取得
files_2014 = os.listdir(DATA_DIR+'/EID34151/myCsv2/2014')
#files_2015 = os.listdir(DATA_DIR+'/EID34151/myCsv/2015')
#files_2016 = os.listdir(DATA_DIR+'/EID34151/myCsv/2016')
#files_2017 = os.listdir(DATA_DIR+'/EID34151/myCsv/2017')

#files = [files_2014, files_2015, files_2016,files_2017]

In [3]:
df = pd.DataFrame()
for file in files_2014:
    csv = pd.read_csv(os.path.join(DATA_DIR,'EID34151/myCsv2/2014',file.replace(".xml.gz",".csv")))
    # -1が入っているところのエラー処理していないことに注意
    df = pd.concat([df, csv[(csv.NumOfTaxIncrease > 0) | (csv.NumOfTaxReduction > 0)]], ignore_index=True)

In [22]:
# 一応保存
#(pd.DataFrame([df.Headline, df.Body]).T).to_csv('../data/EID34151/temp.csv',encoding='utf8',header=True,index=False)

In [95]:
# 切れているけど…
cond = ['ﾄｯﾌﾟ記事:国内ﾆｭｰｽ(日本語)', 'ﾄｯﾌﾟ記事:日本語ﾆｭｰｽ', 'ﾄｯﾌﾟ記事: 海外ﾆｭｰｽ(日本語)']
# コラム、クレジット情報などは一連の記事。分割する必要性？

In [96]:
df2 = pd.DataFrame([df.Headline, df.Body]).T
df2 = df2[df2.Headline.isin(cond)]

In [97]:
df2.head()

,Headline,Body
0,ﾄｯﾌﾟ記事:国内ﾆｭｰｽ(日本語),大納会の東京株式相場は、日経平均株価が９連騰し、連日で年初来高値を更新。為替の円安進行が買い...
1,ﾄｯﾌﾟ記事:国内ﾆｭｰｽ(日本語),大納会の東京株式相場は、日経平均株価が９連騰し、連日で年初来高値を更新。為替の円安進行が買い...
2,ﾄｯﾌﾟ記事:国内ﾆｭｰｽ(日本語),大納会の東京株式相場は、日経平均株価が９連騰し、連日で年初来高値を更新。為替の円安進行が買い...
5,ﾄｯﾌﾟ記事:国内ﾆｭｰｽ(日本語),大納会の東京株式相場は、日経平均株価が９連騰し、連日で年初来高値を更新。為替の円安進行が買い...
29,ﾄｯﾌﾟ記事:国内ﾆｭｰｽ(日本語),大納会の東京株式相場は、日経平均株価が９連騰し、連日で年初来高値を更新。為替の円安進行が買い...


In [102]:
#もう少しきれいになるはず
body_list = list(df2.Body)
all_article_body = []
headline = []
for i in range(len(body_list)):
    temp = body_list[i].split('...')
    for article in temp:
        hoge = article.split('】')
        # タイトルなし
        if (len(hoge)==1):
            all_article_body.append(hoge[-1].replace('\u3000',''))
            headline.append('nan')
        # 一般的なもの
        elif (len(hoge)==2):
            all_article_body.append(hoge[-1].replace('\u3000',''))
            headline.append(hoge[0].split(')')[-1].split('【記者')[0])
        # 【クレジット市場】などがタイトルに
        else:
            all_article_body.append(hoge[-1].replace('\u3000',''))
            headline.append('【'+article.split('【')[1])

In [103]:
all_article

['大納会の東京株式相場は、日経平均株価が９連騰し、連日で年初来高値を更新。為替の円安進行が買い安心感を誘い、経済好転やデフレ脱却への期待などから年明け以降の先高観も強かった。国際原油市況の続伸を受け石油や鉱業、商社株が上昇。不動産や銀行株も高く、電機など輸出関連株も堅調だった。日経平均株価の終値は前週末比112円37銭（0.7％）高の１万6291円31銭。９連騰は、2009年７月以来の連続上昇記録に並ぶ。年間では57％上昇し、上昇率は列島改造ブームに沸いた1972年（92％）以来の大きさとなった。ＴＯＰＩＸは12.22ポイント（0.9％）高の1302.29と４日続伸。年間上昇率は51％と1999年以来、14年ぶりの大きさだ。アムンディ・ジャパンの高野雅永シニアストラテジ',
 '記事全文:{FIFWNSNMYLXE06K50YG<GO>}2)日経平均年間で57％高、41年ぶり上昇率－円安と五輪決定、米景気【記者：河野敏、AnnaKitanaka】12月30日大納会の日経平均株価は前年末比5896円13銭（57％）高の１万6291円31銭と２年連続で上昇し、年間上昇率は1972年（92％）以来、41年ぶりの大きさを記録した。しんきんアセットマネジメント投信の藤原直樹運用部長は、ことしの日本株は、４月の日本銀行による異次元金融緩和を受けた円安効果が大きく、「それがしぼみかけたタイミングでオリンピックが決まり、年末にかけては米国のテーパリング（量的金融緩和の縮小）と経済回復に目が向かった」と指摘。刺激材料の続いたこの１年を振り返った。ことしの日経平均を月ごとに振り返ると、１－４月に４カ月連続で上げ、大発会の１万600円台から１万3000円台まで上昇。前年末に誕生した安倍晋三政権への政策期待が',
 '記事全文:{FIFWNSNMYLVTZ6K50XW<GO>}3)【クレジット市場】黒田緩和で国債損失限定へ、連続プラス逃しても(クレジット市場のニュースアラートはSALTJNCREDIT<GO>、記事はこちらをご覧ください）【記者：野沢茂樹、KevinBuckland、石川茉莉子】12月30日市場関係者は2014年の日本国債の投資収益は円安・株高や景気回復を背景に11年ぶりにマイナスに転じると予想するが、日本銀行が巨額の国債買い入れを進める中、他の主要国に比べると小幅な損失

In [104]:
hogege = pd.DataFrame({'headline':headline, 'body':all_article_body}).drop_duplicates('body')

In [105]:
hogege

,body,headline
0,大納会の東京株式相場は、日経平均株価が９連騰し、連日で年初来高値を更新。為替の円安進行が買い...,nan
1,12月30日大納会の日経平均株価は前年末比5896円13銭（57％）高の１万6291円31銭...,日経平均年間で57％高、41年ぶり上昇率－円安と五輪決定、米景気
2,12月30日市場関係者は2014年の日本国債の投資収益は円安・株高や景気回復を背景に11年ぶ...,【クレジット市場】黒田緩和で国債損失限定へ、連続プラス逃しても(クレジット市場のニュースアラ...
3,12月30日脱デフレを目指すアベノミクスにとって賃上げの拡大が成否を左右する要因となりそうだ...,アベノミクスの成否が来年正念場へ、賃上げペースが課題に－調査
4,12月30日東京外国為替市場では、円が対ドルで約５年２カ月ぶりの安値を更新。米国の景気回復や...,円が対ドルで５年２カ月ぶり安値更新、日米金融政策の違いで先安観
5,12月30日今年の円の対ドル相場は日本銀行による大規模な金融緩和などを背景に34年ぶりの下落...,34年ぶり下落率で円の割高感解消、来年前半に揺り戻しリスク－95円も（通貨コラムのニュースア...
6,12月30日2013年大納会の債券相場は下落。米国債市場で長期金利が上昇したことに加え、国内...,債券は下落、株高・円安や米金利上昇で売り－先物は３カ月半ぶり安値
7,12月30日セブン＆アイ・ホールディングスの村田紀敏社長は、ブルームバーグのインタビューに応...,７＆ｉ社長：今後も米国事業２桁増益目指す－日本のノウハウ応用
8,12月30日ＴＯＰＩＸは年間で51％上昇。*ＴＯＰＩＸを構成する1761銘柄のうち、値上がり...,13年の上昇率１位は日トリム、下落はジーンズ－東証１部
9,12月31日子会社アクリフーズ群馬工場で製造した冷凍食品から農薬「マラチオン」が検出された問...,マルニチＨ：製造工程上で汚染された可能性低いと群馬県－農薬検出
